In [ ]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

# 1. 独热编码

In [ ]:
# 0 表示 vocab 中的第一个词，2 表示第三个词
F.one_hot(torch.tensor([0, 2]), len(vocab))

小批量数据大小形状（批量大小，时间步数）

In [ ]:
# 生成从 0 到 9 的一维张量； 2 行 5 列的矩阵
X = torch.arange(10).reshape((2, 5))
# 转置，变成 5 行 2 列；
# 每个数字都会被编码成长度为 28 的 one-hot 向量
F.one_hot(X.T, 28).shape

# 2. 初始化模型参数


In [ ]:
def get_params(vocab_size, num_hiddens, device):
    """
    vocab_size: 词汇表大小，即 one-hot 编码的长度。
    num_hiddens: 隐藏单元数量，控制模型容量。
    device: 指定运行设备（CPU 或 GPU）。
    """
    # 输入层维度 num_inputs 就是 one-hot 向量的长度
    # = 等于词汇表大小
    # = 输出层维度 num_outputs，因为我们要预测下一个词的概率分布
    num_inputs = num_outputs = vocab_size

    # 生成服从标准正态分布的随机数，并乘以 0.01 进行缩放
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))   # 输入 -> 隐藏
    W_hh = normal((num_hiddens, num_hiddens))  # 隐藏 -> 隐藏（循环）
    b_h = torch.zeros(num_hiddens, device=device)

    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))  # 隐藏 -> 输出
    b_q = torch.zeros(num_outputs, device=device)

    # 附加梯度a
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    # 这些参数需要计算梯度，以便优化器更新它们
    for param in params:
        param.requires_grad_(True)
    return params

# 3. 循环神经网络模型

In [ ]:
# 初始化隐藏状态
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device = device),)

In [ ]:
# RNN 前向传播 rnn
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:
        # 更新隐藏状态
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        # 计算输出
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    # 所有时间步的输出拼接成一个大张量
    # 形状变成 (T*N, V)，方便后续计算 loss
    return torch.cat(outputs, dim=0), (H,)

In [ ]:
# 把前面的函数包装一下
class RNNModelScratch:
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        # 调用前向传播函数
        return self.forward_fn(X, state, self.params)
    # 获取初始状态的方法
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [ ]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

# 4. 预测

In [ ]:
def predict_ch8(prefix, num_preds, net, vocab, device):
    """prefix：是给定句子/词的开头，后面生成新字符"""
    # 1. 初始化隐藏状态
    state = net.begin_state(batch_size=1, device=device)
    # 2. 初始化输出序列（存储字符索引）
    outputs = [vocab[prefix[0]]]
    # 3. 定义输入生成函数（每次取最新输出的索引作为输入）
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    # 4. 预热期（Warm-up）：输入prefix中除第一个字符外的所有字符，更新隐藏状态
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)  # 前向传播计算state
        outputs.append(vocab[y])            # 存储prefix剩余字符的索引
    # 5. 预测期：生成num_preds个新字符
    for _ in range(num_preds):
        y, state = net(get_input(), state) # 前向传播预测下一个字符
        outputs.append(int(y.argmax(dim=1).reshape(1))) # 前向传播预测下一个字符
    # 6. 将索引转为字符并拼接成字符串
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [ ]:
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())


# 5. 梯度裁剪

In [ ]:
def grad_clipping(net, theta):
    """裁剪梯度"""
    # 获取模型参数（兼容PyTorch模块和自定义模型）
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    # 计算所有参数梯度的L2范数
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    # 如果梯度范数超过阈值theta，则进行裁剪
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm # 按比例缩放梯度

# 6. 训练

In [ ]:
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期（定义见第8章）"""
    state, timer = None, d2l.Timer() # 初始化隐藏状态和计时器
    metric = d2l.Accumulator(2)  # 累计损失和词元数量
    for X, Y in train_iter: # 遍历训练数据
        if state is None or use_random_iter:    # 初始或随机抽样时初始化state
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            # 初始或随机抽样时初始化state
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        # 调整标签形状为一维
        y = Y.T.reshape(-1)
        # 数据移至设备
        X, y = X.to(device), y.to(device)
        # 前向传播
        y_hat, state = net(X, state)
        # 损失
        l = loss(y_hat, y.long()).mean()
        # PyTorch优化器
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            # 梯度裁剪
            grad_clipping(net, 1)
            updater.step()
        else:       # 自定义优化器
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel()) # 累计指标
    # 返回困惑度和吞吐量
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [ ]:
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()  # 定义交叉熵损失
    # 初始化可视化动画器
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化优化器
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    # 定义预测函数
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练循环
    for epoch in range(num_epochs):
        # 训练一个epoch
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller')) # 每10轮生成示例文本
            animator.add(epoch + 1, [ppl]) # 更新可视化
    # 最终输出
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))

In [ ]:
num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

检查一下使用随机抽样方法的结果

In [ ]:
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu(),
          use_random_iter=True)